In [4]:
import io
from IPython.nbformat import current

def execute_notebook(nbfile):
    
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    
    ip = get_ipython()
    
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("../workers/Imports.ipynb")
execute_notebook("../workers/scoring_system.ipynb")

Populating the interactive namespace from numpy and matplotlib


//anaconda/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['angle', 'datetime', 'absolute']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


6.6899999999999995

In [5]:
# import individual shooting data
df = pd.read_csv('../../processed_data/identified_shots.csv', index_col=0)
df = df[df.player_id != '(null)']
df.player_id = df.player_id.astype(str)
len(df)

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,8,15,22,23,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
//anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


6727138

### Player Percentages

In [6]:
distance_min, distance_max = 18, 22

df_length = df[(df.shot_distance > distance_min)&(df.shot_distance < distance_max)&(df.depth > -8)&(df.depth < 26)&(df.left_right > -15)&(df.left_right < 15)]
players = list(df_length.player_id.value_counts()[df_length.player_id.value_counts() > 1000].index)
print len(players)

509


In [5]:
player_ids, percentages, player_shots = [], [], []
for player in players:
    player_df = df[(df.player_id == player)&(df.shot_distance > distance_min)&(df.shot_distance < distance_max)]
    player_df['date'] = player_df.shot_date.apply(get_date)
    player_df = player_df.sort_values('shot_date')
    player_df['floor_position_x_feet_shifted'] = player_df.floor_position_x_feet.shift(-1)
    player_df['floor_position_y_feet_shifted'] = player_df.floor_position_y_feet.shift(-1)
    player_df['x_diff'] = abs(player_df.floor_position_x_feet - player_df.floor_position_x_feet_shifted)
    player_df['y_diff'] = abs(player_df.floor_position_y_feet - player_df.floor_position_y_feet_shifted)
    player_df['movement'] = player_df[['x_diff', 'y_diff']].apply(pathag, axis=1)
    player_df_restricted = player_df[(player_df.movement < 2)&(player_df.depth > -8)&(player_df.depth < 26)&(player_df.left_right > -15)&(player_df.left_right < 15)]
    player_average = player_df_restricted.numeric_make_miss.mean()
    player_ids.append(player)
    percentages.append(player_average)
    player_shots.append(len(player_df_restricted))
    print player, len(player_df_restricted), player_average

# make dataframe and save it
player_percentages = pd.DataFrame({'Player': player_ids, 'Number_of_shots': player_shots, 'Percentage': percentages})
player_percentages.to_csv('../../processed_data/players_percentages.20_24.csv')

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


131 21348 0.725079632752
15801 17746 0.587230925279
11105 17196 0.501802744824
3512 12916 0.654769278414
14779 597 0.564489112228
3505 11452 0.664338106881
3517 11084 0.571725009022
3515 9694 0.647720239323
15985 9077 0.572105321141
3489 9564 0.611773316604
3516 8402 0.678409902404
20319 8506 0.73324711968
3514 8476 0.580226521944
3506 7974 0.650238274392
15103 6431 0.56880733945
126 5657 0.606151670497
156 6394 0.565530184548
135 6482 0.671860536871
17373 6731 0.421185559352
20384 6291 0.605309171833
3508 6800 0.518823529412
232 5618 0.543431826273
17683 6224 0.427538560411
17508 5457 0.559464907458
134 4760 0.580672268908
15683 5783 0.568908870828
16645 5420 0.683025830258
132 5153 0.706190568601
16084 4902 0.602407180743
21154 3651 0.551903588058
20269 5084 0.631785995279
3343 5052 0.591448931116
20488 4555 0.549286498353
136 4155 0.593501805054
20336 3314 0.501508750754
3504 4265 0.522626025791
14151 4699 0.751223664609
20240 4441 0.677099752308
20242 4427 0.62706121527
17703 4286 

### Sessions dataframe for learning

In [7]:
sample_sessions = []
# all players
for player in players:
    print player
    player_df_by_session, player_average = get_player_df_by_session(player, distance_min, distance_max, 50)
    print len(player_df_by_session)
    for session in player_df_by_session:
        stats = scores_with_correlations(session)
        session_dictionary = {'sample_percentage': stats[0],
                                'Angle_Avg': stats[1],
                                'Angle_Cons': stats[2],
                                'Depth_Avg': stats[3],
                                'Depth_Cons': stats[4],
                                'Left_Right_Avg': stats[5],
                                'Left_Right_Cons': stats[6],
                                'Angle_depth': stats[7],
                                'Angle_left_Right': stats[8],
                                'Depth_left_right': stats[9],
                                'practice_percentage': player_average,
                                'player_id': player}
        sample_sessions.append(session_dictionary)
session_df = pd.DataFrame(sample_sessions)

11105


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


489
15801
446
131
238
10647
276
15103
62
3489
233
14779
0
20071
188
3512
160
15985
133
3517
143
3516
126
10001
151
3505
91
3506
124
10646
120
15812
87
10943
88
14228
92
135
72
11086
85
3768
6
15683
77
16109
90
127
49
21154
14
14988
89
20675
64
20494
77
17683
80
126
25
16645
45
156
32
132
56
232
42
3514
59
20827
13
20162
48
299
54
16791
0
11087
63
15251
64
3508
78
16984
33
18254
17
16750
30
17508
45
10447
10
20769
73
12752
0
10838
37
11552
73
3999
59
10144
53
19572
63
20140
39
16080
55
134
23
14127
64
17322
64
20269
39
14853
29
3513
63
16110
59
17826
0
9687
18
3952
43
17703
58
20336
3
20384
43
10558
50
17705
56
20915
32
9868
18
20488
44
20451
20
19026
20
17397
12
3509
52
20242
20
12423
43
9048
43
17373
34
10649
21
10196
39
18137
0
13984
13
10814
14
16084
33
30
28
15136
52
4071
34
20901
0
14778
26
20078
20
16331
19
19289
0
9766
50
11080
42
3504
16
16227
6
5636
43
18222
23
15700
47
14816
10
12695
36
18147
25
5408
19
19227
22
12845
0
20061
25
14436
9
16503
2
16547
42
13667
43
20572
40
1095

In [8]:
len(session_df)

12145

In [9]:
def get_correlation(x):
    return x[1]
session_df.Angle_depth = session_df.Angle_depth.apply(get_correlation)

In [10]:
session_df.Angle_left_Right = session_df.Angle_left_Right.apply(get_correlation)
session_df.Depth_left_right = session_df.Depth_left_right.apply(get_correlation)

In [13]:
# save dataframe
session_df.to_csv('../../processed_data/session_df.20_24.movement_2.50.csv')